In [1]:
# 获取当前工作目录

import os

current_directory = os.getcwd()

print("当前工作目录:", current_directory)

当前工作目录: D:\Data\Shap


In [2]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
          
from sklearn.model_selection import train_test_split 


In [3]:
# 设置字体为新罗马数字

plt.rcParams['font.family'] = 'Times New Roman'

# 处理负号 

plt.rcParams['axes.unicode_minus'] = False

# 忽略警告

import warnings
          
warnings.filterwarnings("ignore")

In [4]:
# 读取数据

df = pd.read_csv('Data_target.csv')

# 删除 NaN 值

df.dropna(inplace=True)

In [5]:
# 划分特征和目标变量 

# 删除 'target' 列

X = df.drop(['Target'], axis=1)        


y = df['Target']   

In [6]:
# 划分训练集和测试集  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    
                                                    random_state=42, 
                                                    
                                                    stratify=df['Target'])
# 观察前几行

df.head()

,Size,DOI,TT,TB,BASO%,NLR,Grade,PNI,LVI,Target
0,15,0.2,10.61,1,0.8,2.23,2,0,0,0
1,30,0.3,12.30,1,0.9,2.56,2,1,0,0
2,15,0.4,13.71,29,0.5,3.82,2,1,0,1
3,5,0.5,4.10,7,0.5,2.65,1,0,0,0
4,18,0.6,6.52,2,0.7,0.81,2,0,0,1


In [7]:
from sklearn.neural_network import MLPClassifier

# 模型的构建与训练

model_MLP = MLPClassifier(random_state = 4231)

model_MLP.fit(X, y)

MLPClassifier(random_state=4231)

In [8]:
import shap

# 创建Explainer
explainer = shap.KernelExplainer(model_MLP.predict,X)

# 以numpy数组的形式输出SHAP值

shap_values = explainer.shap_values(X)

# 以SHAP的Explanation对象形式输出SHAP值

shap_obj = explainer(X)


Using 554 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


  0%|          | 0/554 [00:00<?, ?it/s]

  0%|          | 0/554 [00:00<?, ?it/s]

In [9]:
# 绘制 SHAP 蜂群图，使用宽度 6英寸，高度 8 英寸，分辨率为1200
plt.figure(figsize=(6, 8), dpi=1200)

# 绘制 SHAP 蜂群图，并设置 show=False 以避免直接显示
shap.summary_plot(shap_obj.values, X, show=False)

# 开启网格线
plt.grid(visible=True, linestyle='--', color='gray', alpha=0.1)

# 设置坐标轴标签
plt.xlabel('SHAP value (impact on AHD)', fontsize=12, fontweight='bold')
plt.ylabel('', fontsize=10, fontweight='bold')  # 修改y轴标签为 'Features'

# 设置坐标轴刻度标签的字体大小
plt.tick_params(axis='both', labelsize=10)  # 设置x轴和y轴的刻度标签字体大小

# 设置标题
plt.title('', loc='left', fontsize=14, fontweight='bold')

# 保存图像为 PDF 文件
plt.savefig("Shap_Beeswarm_MLP.pdf", format='pdf', bbox_inches='tight')

# 自动调整布局以确保标题显示在图内
plt.tight_layout()

# 显示图片
plt.show()


In [10]:
from matplotlib.colors import LinearSegmentedColormap

# 设置最大显示的特征数
max_display = 10

# 创建图形
plt.figure(figsize=(6, 8), dpi=1200)

# 绘制 SHAP 条形图并隐藏显示（使用 show=False）
shap.summary_plot(shap_obj.values, X, plot_type="bar", show=False)

# 获取当前图形的 Axes
ax = plt.gca()

# 隐藏条形图上的数值
for text in ax.texts:
    text.set_visible(False)

# 获取 SHAP 值并排序
shap_values_sorted = shap_obj.values.flatten()
sorted_indices = np.argsort(shap_values_sorted)[::-1]
sorted_shap_values = shap_values_sorted[sorted_indices][:max_display]

# 创建从橙色到红色的渐变色，并反转顺序
colors = LinearSegmentedColormap.from_list("orange_to_red", ["orange", "red"])(np.linspace(0, 1, max_display))

# 应用颜色到条形图
for i, bar in enumerate(ax.patches):
    if i < max_display:  # 防止颜色超出范围
        bar.set_facecolor(colors[i])

# 添加坐标轴标签
plt.xlabel('Mean SHAP value (average impact on OLNM)', fontsize=10, fontweight='bold')
plt.ylabel('', fontsize=10, fontweight='bold')

# 设置坐标轴刻度标签的字体大小
plt.tick_params(axis='both', labelsize=8)

# 设置标题
plt.title('', loc='left', fontsize=10, fontweight='bold')

# 保存图像为 PDF 文件
plt.savefig("Shap_Bar_MLP.pdf", format='pdf', bbox_inches='tight')

# 自动调整布局
plt.tight_layout()

# 显示图像
plt.show()
